In [ ]:
import torch
import open3d as o3d
import numpy as np
from DCP_model import DCP

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Load point clouds
scan_knee = o3d.io.read_point_cloud("pc_volumes/scan_knee_edited1.ply")
CT_knee = o3d.io.read_point_cloud("pc_volumes/CT_knee_400.ply")

In [ ]:
# Convert Open3D point clouds to NumPy arrays
def o3d_to_numpy(pcd):
    return np.asarray(pcd.points)

#Ensure equal number of points
num_points = 1024  # Adjust if needed
scan_knee = scan_knee.farthest_point_down_sample(num_points)
CT_knee = CT_knee.farthest_point_down_sample(num_points)

scan_knee_np = o3d_to_numpy(scan_knee)
CT_knee_np = o3d_to_numpy(CT_knee)


In [4]:
src = torch.tensor(scan_knee_np, dtype=torch.float32).cuda().unsqueeze(0)
tgt = torch.tensor(CT_knee_np, dtype=torch.float32).cuda().unsqueeze(0)
print(src.size())
print(tgt.size())

torch.Size([1, 1024, 3])
torch.Size([1, 1024, 3])


In [5]:
src = src.transpose(1, 2)  # Shape: (1, 3, 1024)
tgt = tgt.transpose(1, 2)  # Shape: (1, 3, 1024)
print(src.size())
print(tgt.size())

torch.Size([1, 3, 1024])
torch.Size([1, 3, 1024])


In [6]:
'''torch.backends.cudnn.deterministic = True
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)'''

'torch.backends.cudnn.deterministic = True\ntorch.manual_seed(42)\ntorch.cuda.manual_seed_all(42)\nnp.random.seed(42)'

In [7]:
# Load a pre-trained DCP model
dcp_checkpoint = "dcp/pretrained/dcp_v1.t7"
dcp_model = DCP().cuda()
dcp_model.load_state_dict(torch.load(dcp_checkpoint), strict=False)  # Load trained model weights
dcp_model.eval()

C:\Users\ge75wix\AppData\Local\Temp\ipykernel_18648\1870614810.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dcp_model.load_state_dict(torch.load(dcp_checkpoint), stri

DCP(
  (emb_nn): DGCNN(
    (conv1): Conv2d(6, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv3): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv4): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv5): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn5): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pointer): Transformer(
    (model): EncoderDecoder(
      (encoder): Encoder(
        (layers): ModuleList(
          (0): EncoderLa

In [8]:
rotation_ab, translation_ab, rotation_ba, translation_ba = dcp_model(src, tgt)

In [9]:
rotation_ab = rotation_ab.detach().cpu().numpy()
translation_ab = translation_ab.detach().cpu().numpy()
rotation_ba = rotation_ba.detach().cpu().numpy()
translation_ba = translation_ba.detach().cpu().numpy()

In [10]:
print(rotation_ba)
print(translation_ba)

[[[ 0.67766327 -0.14930981 -0.72005486]
  [-0.7209557   0.05804037 -0.6905461 ]
  [ 0.1448975   0.9870856  -0.06831388]]]
[[ 235.13379    59.301376 -324.9567  ]]


In [ ]:
# Load the point clouds
scan_knee_1 = o3d.io.read_point_cloud("pc_volumes/scan_knee_edited1.ply")
CT_knee_1 = o3d.io.read_point_cloud("pc_volumes/CT_knee_400.ply")

# Create a 4x4 transformation matrix
transformation_matrix = np.eye(4)
transformation_matrix[:3, :3] = rotation_ba
transformation_matrix[:3, 3] = translation_ba.flatten()

# Apply the transformation to the scan_knee point cloud
CT_knee_transformed = CT_knee_1.transform(transformation_matrix)

# Visualize both point clouds in the same window
CT_knee_transformed.paint_uniform_color([1, 0, 0])  # Color the transformed scan_knee red
scan_knee_1.paint_uniform_color([0, 1, 0])  # Color the CT_knee green

# Create a visualizer and add both point clouds
vis = o3d.visualization.Visualizer()
vis.create_window()

vis.add_geometry(CT_knee_transformed)
vis.add_geometry(scan_knee_1)

#vis.capture_screen_image('testlast.png', True)


# Run the visualizer
vis.run()
'''
# Get the point clouds from the scene
pcd_combined = o3d.geometry.PointCloud()
pcd_combined += CT_knee_transformed
pcd_combined += scan_knee_1

# Save the point cloud
o3d.io.write_point_cloud("pc_volumes/output.ply", pcd_combined)'''

vis.destroy_window()

'# Create transformation matrices\ntransformation_ab = np.eye(4)\ntransformation_ab[:3, :3] = rotation_ab  # Apply rotation_ab\ntransformation_ab[:3, 3] = translation_ab.flatten()  # Apply translation_ab\n\ntransformation_ba = np.eye(4)\ntransformation_ba[:3, :3] = rotation_ba  # Apply rotation_ba\ntransformation_ba[:3, 3] = translation_ba.flatten()  # Apply translation_ba\n\n# Transform the point clouds\nscan_knee_transformed = scan_knee_1.transform(transformation_ab)  # Align scan_knee to CT_knee\nCT_knee_transformed = CT_knee_1.transform(transformation_ba)  # Align CT_knee to scan_knee\n\n# Color the point clouds for distinction\nscan_knee_transformed.paint_uniform_color([1, 0, 0])  # Red for scan_knee\nCT_knee_transformed.paint_uniform_color([0, 1, 0])  # Green for CT_knee\n\n# Create a visualizer\nvis = o3d.visualization.Visualizer()\nvis.create_window()\n\n# Add both transformed point clouds to the visualizer\nvis.add_geometry(scan_knee_transformed)\nvis.add_geometry(CT_knee_tran